In [3]:
#!pip install torch
#!pip install transformers

In [13]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch

In [12]:
import pickle

embeddings = []

for i in range(0, 501):
    with open(f"embeddings/sbert_embeddings{i}.json", "rb") as file:
        obj = pickle.load(file)
        embeddings.extend(obj)

768


In [61]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sfurkan/LexBERT-turkish-uncased')

No sentence-transformers model found with name C:\Users\Furkan/.cache\torch\sentence_transformers\sfurkan_LexBERT-turkish-uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\Furkan/.cache\torch\sentence_transformers\sfurkan_LexBERT-turkish-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification 

In [51]:
from tqdm import tqdm

TARGETED_DATASET_PATH = r'data_text.csv'

texts = pd.read_csv(TARGETED_DATASET_PATH)["data_text"]

sentenceIndex2DocIndex = {}
lastSentenceIndex = 0
currentDocIndex = 0
for text in tqdm(texts):
    sentences = text.strip().split('.')
    
    for idx, sentence in enumerate(sentences):
        sentenceIndex2DocIndex[idx + lastSentenceIndex] = currentDocIndex
    
    lastSentenceIndex += len(sentences)
    currentDocIndex += 1

100%|████████████████████████████████████████████████████████████████████████████| 4142/4142 [00:00<00:00, 8908.17it/s]


In [207]:
inputEmbedding = model.encode('''
suretiyle; çalışmaya başladığı gün iş kazasına uğraması halinde ise aynı veya emsal işte 
çalışan benzeri bir sigortalının günlük kazancı esas tutulacaktır.

4 üncü maddenin birinci fıkrasının (a) bendi gereği sigortalı sayılanların ödenek 
veya gelire esas günlük kazançlarının hesabında:

a-Prim, ikramiye ve bu nitelikteki arızi ödemeler dikkate alınmış ise ödenek ve gelire 
esas günlük kazanç, ücret toplamının ücret alınan gün sayısına bölünmesiyle hesaplanacak 
günlük kazanca, % 50 oranında bir ekleme yapılarak bulunan tutardan çok olamaz.

b-İdare veya yargı mercilerince verilen karar gereğince yapılan ücret, ikramiye, zam, 
tazminat ve bu mahiyetteki ödemelerden, ödenek ve gelirin hesabına esas alman üç aylık 
dönemden önceki aylara ilişkin olanlar dikkate alınmaz.

Meslek hastalığı, sigortalının sigortalı olarak çalıştığı son işinden ayrıldığı tarihten bir 
yıl geçtikten sonra meydana çıkmış ise, günlük kazancı bu son işinden ayrıldığı tarih esas 
alınarak yukarıdaki fıkralara göre hesaplanır.

Kanunun 4 üncü maddesinin birinci fıkrasının (b) bendi kapsamındaki sigortalılar için 
aylık prime esas kazanç, Kanunun 82 nci maddesine göre belirlenen prime esas günlük kazanç 
alt sının ile üst sının arasında kalmak şartı ile kendileri tarafından beyan edilecek günlük 
kazancın otuz katı olduğundan, geçici iş göremezlik ödeneğinde esas alınacak tutarlann 
hesabı buna göre yapılacaktır.

Sigortalılara hesaplanacak günlük kazancının yatarak tedavilerde yansı, ayaktan 
tedavilerde ise üçte ikisi (2/3) üzerinden hesap edilerek geçici iş göremezlik ödeneği 
verilecektir.

Geçici iş göremezlik ödenek hesabına esas tutulan devrede prim ve ikramiye gibi 
ödemelerin bulunması halinde, Kanunun 17 nci maddesinin üçüncü fıkrasının (a) bendine 
göre prim, ikramiye ve bu nitelikteki anzi ödemeler dikkate alınmış ise, ödenek ve gelire esas 
alınacak günlük kazanç ücret toplamının ücret alman gün sayısına bölünmesiyle hesaplanacak 
günlük kazanca % 50 oranında bir ekleme yapılarak bulunan tutardan çok olamayacağından 
geçici iş göremezlik ödeneği şu şekilde hesaplanacaktır.

Ornek-l:Nisan 2016 tarihinde hastalığı nedeniyle istirahatli bırakıldığı için işyerinde 
çalışamamış olan ve 01/04/2016 tarihinde iş kazası geçirmesi nedeniyle istirahatli bırakılan 
sigortalıya aşağıda belirtilen prim ödeme gün sayılan ile kazançlanna göre, verilecek geçici iş 
göremezlik ödeneğine esas tutulacak günlük kazancı hesaplanacaktır.
''')

In [208]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

documentEmbeddings = []
embeddingIndex = 0
for idx, text in tqdm(enumerate(texts)):
    sents = text.strip().split('.')
    sentEmbeddings = []
    for sent in sents:
        sentEmbeddings.append(embeddings[embeddingIndex])
        embeddingIndex += 1
    documentEmbeddings.append(sentEmbeddings)


documentScores = []    
for docEmb in documentEmbeddings:
    similarities = cosine_similarity(docEmb, [inputEmbedding])
    meanSim = 0
    for sim in similarities:
        meanSim += sim[0]
    meanSim /= len(similarities)
    documentScores.append(meanSim)

maxSim = -1
maxIndex = -1
best = []
for idx, docScore in enumerate(documentScores):
    if docScore > maxSim:
        maxSim = docScore
        maxIndex = idx
        best.append((docScore, texts[idx]))

4142it [00:00, 9453.68it/s] 


In [209]:
print(sorted(best, key=lambda x: x[0])[:10])

[(0.7648004279471934, 'Resmî Gazete Tarihi: 11.07.2012 Resmî Gazete Sayısı: 28350\nKanun No. 6338 Kabul Tarihi : 29/06/2012\nBÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İLGİLİ KANUN VE KANUN HÜKMÜNDE KARARNAMELERE EKLENMESİNE DAİR KANUN\n\nMadde 1 – 5/1/1961 tarihli ve 237 sayılı Taşıt Kanununun 10 uncu maddesine aşağıdaki fıkralar eklenmiştir.\n\n“Türk Silahlı Kuvvetleri (Jandarma Genel Komutanlığı ve Sahil Güvenlik Komutanlığı dâhil), Emniyet Genel Müdürlüğü ile Gümrük ve Ticaret Bakanlığı Gümrükler Muhafaza Genel Müdürlüğüne kurum, kuruluş, dernek ve vakıflarca hibe edilecek taşıtlar, merkezî yönetim bütçe kanununa bağlı (T) işaretli cetvelde gösterilmesine gerek bulunmaksızın Bakanlar Kurulu kararı ile edinilebilir.\n\nEmniyet Genel Müdürlüğüne ait taşıtlar, 12/4/2001 tarihli ve 4645 sayılı Emniyet Genel Müdürlüğüne Ait Araç, Gereç, Mal ve Malzemenin Satış, Hibe, HEK ve Hurda Durum ve İşlemleri ile Hizmet Satışına Dair Kanun hükümleri çerçevesinde merkezî yönetim bütçe kanununa bağ